In [1]:
import os
os.chdir('..')

### Prepare config

In [5]:
from src.config import Config

config = Config()
config.read_config('config/train.config')
status = config.status.lower()

def data_initialization(config):
#     config.initial_feature_alphabets()
    config.build_alphabet(config.train_dir)
    config.build_alphabet(config.dev_dir)
    config.build_alphabet(config.test_dir)
    config.fix_alphabet()

data_initialization(config)

### Debug `read_instance`

In [35]:
import json

def normalize_word(word):
    new_word = ""
    for char in word:
        if char.isdigit():
            new_word += '0'
        else:
            new_word += char
    return new_word


def read_instance(input_file, word_alphabet, char_alphabet, feature_alphabets, label_alphabet, number_normalized, max_sent_length, split_token='\t', char_padding_size=-1, char_padding_symbol = '</pad>'):
    feature_num = len(feature_alphabets)
    # in_lines = open(input_file,'r', encoding="utf8").readlines()
    with open(input_file, 'rb') as f:
        in_lines = f.readlines()
    in_lines = [json.loads(line.decode('utf-8')) for line in in_lines]
    instance_texts = []
    instance_Ids = []
    words = []
    labels = []
    word_Ids = []
    label_Ids = []
    doc_id = ""
    # else:
    ### for sequence labeling data format i.e. CoNLL 2003
    for line in in_lines:
        doc_id = line['doc_id']
        seq = line['seq']

        # JT: iterate thru seq for word and labels
        for word, label, word_idx in seq:
            words.append(word)
            if number_normalized:
                word = normalize_word(word)
            labels.append(label)
            word_Ids.append(word_alphabet.get_index(word))
            label_Ids.append(label_alphabet.get_index(label))

        # end of seq
        period_id = word_alphabet.get_index(".")
        sent_word_Ids_list = []
        idx = 0
        sent_word_Ids = []
        while idx <= len(word_Ids) - 1:
            sent_word_Ids.append(word_Ids[idx])
            if word_Ids[idx] == period_id:
                sent_word_Ids_list.append(sent_word_Ids)
                sent_word_Ids = []
            idx += 1
        if sent_word_Ids:
            sent_word_Ids_list.append(sent_word_Ids)
        if len(words)>0:
            instance_texts.append([words, labels, doc_id])
            instance_Ids.append([word_Ids, sent_word_Ids_list, label_Ids])
        words = []
        labels = []
        word_Ids = []
        label_Ids = []

    #         # if not doc_id: 
    #         #     doc_id = line.strip()
    #         #     continue
    #         if len(line) > 2:
    #             pairs = line.strip().split()
    #             word = pairs[0]
    #             if sys.version_info[0] < 3:
    #                 word = word.decode('utf-8')
    #             words.append(word)
    #             if number_normalized:
    #                 word = normalize_word(word)
    #             label = pairs[-1]
    #             labels.append(label)
    #             word_Ids.append(word_alphabet.get_index(word))
    #             label_Ids.append(label_alphabet.get_index(label))
    #         else:
    #             if (len(words) > 0) and ((max_sent_length < 0) or (len(words) < max_sent_length)) :
    #                 # get sent_word_Ids_list (split with ".")
    #                 period_id = word_alphabet.get_index(".")
    #                 sent_word_Ids_list = []
    #                 idx = 0
    #                 sent_word_Ids = []
    #                 while idx <= len(word_Ids) - 1:
    #                     sent_word_Ids.append(word_Ids[idx])
    #                     if word_Ids[idx] == period_id:
    #                         sent_word_Ids_list.append(sent_word_Ids)
    #                         sent_word_Ids = []
    #                     idx += 1
    #                 if sent_word_Ids:
    #                     sent_word_Ids_list.append(sent_word_Ids)

    #                 instance_texts.append([words, labels, doc_id])
    #                 instance_Ids.append([word_Ids, sent_word_Ids_list, label_Ids])
    #                 # import ipdb; ipdb.set_trace()

    #             words = []
    #             labels = []
    #             word_Ids = []
    #             label_Ids = []
    #             # doc_id = ""
    # if (len(words) > 0) and ((max_sent_length < 0) or (len(words) < max_sent_length)) :
    #     instance_texts.append([words, labels, doc_id])
    #     # instance_Ids.append([word_Ids, label_Ids])
    #     instance_Ids.append([word_Ids, sent_word_Ids_list, label_Ids])
    #     words = []
    #     labels = []
    #     word_Ids = []
    #     label_Ids = []
    #     doc_id = ""
    return instance_texts, instance_Ids

config.train_texts, config.train_Ids = read_instance(config.train_dir, config.word_alphabet, config.char_alphabet, config.feature_alphabets, config.label_alphabet, config.number_normalized, config.MAX_SENTENCE_LENGTH, config.split_token)

In [36]:
config.train_texts[511]

[['as',
  'detailed',
  'by',
  'microsoft',
  ':',
  'this',
  'new',
  'feature',
  'is',
  'only',
  'available',
  'for',
  'windows',
  'insider',
  '##s',
  'running',
  'windows',
  '10',
  'insider',
  'preview',
  ',',
  'build',
  '1835',
  '##1',
  'or',
  'later',
  '.',
  'this',
  'version',
  'of',
  'windows',
  '10',
  'has',
  'not',
  'yet',
  'been',
  'released',
  'publicly',
  '.'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'bleeping-71']

In [37]:
for idx, id_ in enumerate(config.train_Ids):
    number_data = len(id_[0])
    print(idx, number_data, config.train_texts[idx][-1] )
    assert number_data>0

0 105 fireeye-1
1 57 fireeye-1
2 35 fireeye-1
3 99 fireeye-2
4 51 fireeye-2
5 32 fireeye-2
6 50 fireeye-3
7 68 fireeye-3
8 61 fireeye-3
9 45 fireeye-3
10 97 fireeye-4
11 134 fireeye-4
12 103 fireeye-4
13 81 fireeye-4
14 102 fireeye-5
15 88 fireeye-5
16 45 fireeye-5
17 93 fireeye-6
18 89 fireeye-6
19 56 fireeye-6
20 35 fireeye-6
21 66 fireeye-8
22 62 fireeye-8
23 50 fireeye-8
24 65 fireeye-9
25 52 fireeye-9
26 32 fireeye-9
27 28 fireeye-9
28 108 fireeye-10
29 99 fireeye-10
30 63 fireeye-10
31 75 fireeye-11
32 51 fireeye-11
33 135 fireeye-12
34 93 fireeye-12
35 60 fireeye-12
36 41 fireeye-12
37 129 fireeye-13
38 45 fireeye-13
39 108 fireeye-15
40 80 fireeye-15
41 48 fireeye-15
42 89 fireeye-17
43 58 fireeye-17
44 30 fireeye-17
45 134 fireeye-18
46 116 fireeye-18
47 77 fireeye-18
48 47 fireeye-18
49 121 fireeye-21
50 131 fireeye-21
51 80 fireeye-21
52 40 fireeye-21
53 59 fireeye-22
54 22 fireeye-22
55 13 fireeye-22
56 118 fireeye-23
57 77 fireeye-23
58 136 fireeye-24
59 95 fireeye-24
60 3

762 93 bleeping-94
763 105 bleeping-100
764 61 bleeping-43
765 48 bleeping-25
766 27 bleeping-26
767 109 bleeping-15
768 5 bleeping-55
769 51 bleeping-55
770 29 bleeping-21
771 93 bleeping-26
772 46 fireeye-30
773 151 bleeping-30
774 170 bleeping-61


In [28]:
config.train_Ids[511 ]

[[], [], []]

In [16]:
dir(config.word_alphabet)

['UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add',
 'clear',
 'close',
 'default_index',
 'enumerate_items',
 'from_json',
 'get_content',
 'get_index',
 'get_instance',
 'instance2index',
 'instances',
 'iteritems',
 'keep_growing',
 'label',
 'load',
 'name',
 'next_index',
 'open',
 'save',
 'size']

In [21]:
config.word_alphabet.get_index('at')

345